In [9]:
import json
import cohere
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

load_dotenv()

/home/codespace/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [7]:


documents = []

with open('documents.json', 'r') as file:

    docs = json.load(file)
    
    

In [8]:
for doc in docs:

    for document in doc['documents']:

        document['course'] = doc['course'] 
        documents.append(document)

In [4]:
es_client = Elasticsearch("http://localhost:9200")

In [5]:
es_client.info()

{'name': '7063d1da7e3a',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'gAyh2ee3QrKL9H8JPs0txg',
 'version': {'number': '8.4.3',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73',
  'build_date': '2022-10-04T07:17:24.662462378Z',
  'build_snapshot': False,
  'lucene_version': '9.3.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [6]:
index_settings = {"settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "faq_search"

es_client.indices.create(index = index_name, body = index_settings)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'faq_search'}

In [16]:
for doc in documents:
    
    es_client.index(index = index_name, body=doc)

In [27]:
def search(query, coursefilter = "data-engineering-zoomcamp"):

    search_settings =  {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": coursefilter
                    }
                }
            }
        }
    }

    response = es_client.search(index = index_name, body = search_settings)

    result_docs = []
   
    #print(response)
    for hit in response['hits']['hits']:

        result_docs.append(hit['_source'])
    
    return result_docs

In [20]:
def build_prompt(query, related_docs):
    

    prompt = """
    You are an assistant for teaching online courses answer the question based on the context provided.
    Only the information available in the provided context should be used strictly adhere to this
    
    question:{query}
    
    context:{context}
    
    
    """.strip()
    
    
    context = ""
    for doc in related_docs:
        
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']} \n\n"
        
    
    prompt = prompt.format(query = query, context = context).strip()
    
    return prompt

In [21]:
def chat(prompt, model = "command-r-plus"):
    co_client = cohere.Client(api_key = os.environ["KEY"])
    
    response = co_client.chat( model = model, message = prompt)
    
    return response.text

In [24]:
def rag(query, model = "command-r-plus"):

    results = search(query = query)
    prompt = build_prompt(query = query, related_docs = results)

    answer = chat(prompt, model = model)

    return answer

In [25]:
query = "I just found out the course has started can i still join it ?"
answer = rag( query = query)

{'took': 26, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 413, 'relation': 'eq'}, 'max_score': 24.283089, 'hits': [{'_index': 'faq_search', '_id': '7seUB5EBZo1SkcPJluk3', '_score': 24.283089, '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}}, {'_index': 'faq_search', '_id': '88eUB5EBZo1SkcPJlunG', '_score': 18.019043, '_source': {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also s

In [26]:
answer

'Yes, you can still join the course after the start date. You are eligible to submit homework, but be mindful of the deadlines for turning in final projects.'